In [1]:
!mkdir checkpoints

In [ ]:
!pip3 install speechtokenizer

  Using cached speechtokenizer-1.0.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached lion_pytorch-0.2.3-py3-none-any.whl.metadata (616 bytes)
  Using cached contourpy-1.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (101 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp39-cp

In [2]:
!wandb login

wandb: Currently logged in as: irodkin to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:

!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: Can't create temporary cache file /etc/ld.so.cache~: Permission denied


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from speechtokenizer import SpeechTokenizer
import soundfile as sf
import torchaudio
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_QUANTIZERS_USED = 4
batch_size = 4

config_path = '../SpeechTokenizer/speechtokenizer_hubert_avg/config.json'
ckpt_path = '../SpeechTokenizer/speechtokenizer_hubert_avg/SpeechTokenizer.pt'
speech_tokenizer = SpeechTokenizer.load_from_checkpoint(config_path, ckpt_path).to(device)
speech_tokenizer.eval()


def normalize_waveform(waveform, sr):
    if len(waveform.shape) == 2 and waveform.shape[1] > 0:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.reshape(1, -1)
    waveform = torchaudio.functional.resample(waveform, sr, speech_tokenizer.sample_rate)
    return waveform


# Expected input: waveform of shape (B, C, V)
# The encoding returns shape (num codebooks, 1, timestep). Using it with batch size > 1 gave me a lot of errors so I didn't try it.
# After encoding the tokens, I threw away some of the later codebooks which encode less information before I flattened them as explained below.
def tokenize(waveform):
    with torch.no_grad():
        codes = speech_tokenizer.encode(waveform.unsqueeze(0).to(device))  # codes: (n_q, B, T)
    semantic_tokens = codes[:NUM_QUANTIZERS_USED, :, :].cpu()
    return flatten_tokens(semantic_tokens)

def save_waveform(filename, waveform):
  torchaudio.save(filename, waveform[0].detach().cpu(), 16000)

def decode_tokens(tokens):
  unflattened_tokens = unflatten_tokens(tokens)
  return speech_tokenizer.decode(unflattened_tokens)

def save_to_file(tok, filename):
  outputwav = decode_tokens(tok.detach().to(device))
  save_waveform(filename, outputwav)

# Transposing the timestep and code books before flattening to have it be a1, b1, c1 instead of a1, a2, a3, b1, b2, b3,
# since I'm throwing away some of the codebooks, and I also want to be able to generate based on timestep
def flatten_tokens(tokens):
    n_q, B, T = tokens.shape
    transpose_tokens = tokens.transpose(0, 2)
    return transpose_tokens.reshape(B, T * NUM_QUANTIZERS_USED)


def unflatten_tokens(tokens):
    B, L = tokens.shape
    T = L // NUM_QUANTIZERS_USED
    return tokens.reshape(T, B, NUM_QUANTIZERS_USED).transpose(0, 2)



/home/ivan.rodkin/miniconda3/envs/speech/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ivan.rodkin/miniconda3/envs/speech/lib/python3.9/site-packages/speechtokenizer/model.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

In [4]:
from pytubefix import Playlist

def download_audio_from_playlist(playlist_url, output_path):
    playlist = Playlist(playlist_url)
    for video in playlist.videos:
        try:
            audio_stream = video.streams.get_audio_only()
            audio_stream.download(output_path=output_path, filename=video.title + ".mp4")
        except Exception as e:
            pass
# Using SolidJJ Stooge for fun
playlist_url = "https://youtube.com/playlist?list=PLIIoO74XWQWqox6Nn2HEm5Y0_fWJ9XDOu&si=AMWyAMNcDrA0Hqt_"
download_audio_from_playlist(playlist_url, 'audio/')

In [5]:
from moviepy.editor import AudioFileClip
import os
# Length of the clips for training
LENGTH_OF_CLIPS = 10

def convert_mp4_to_wav_clips(mp4_file, output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  # Convert mp4 to wav and split into 10-second clips
  # Load the audio from the video file
  audio = AudioFileClip(mp4_file)

  # Duration of the audio in seconds
  duration = int(audio.duration)

  # Split the audio into 10-second clips and save
  mp4_output_name = os.path.basename(mp4_file).split('.')[0].replace(' ', '_').replace("#", "num")
  print(mp4_output_name)
  for start in range(0, duration, LENGTH_OF_CLIPS):
      outputpath = os.path.join(output_dir, f'{mp4_output_name}_clip_{start}_{start+LENGTH_OF_CLIPS}.wav')
      if os.path.exists(outputpath):
          continue
      end = min(start + LENGTH_OF_CLIPS, duration)
      clip = audio.subclip(start, end)
      clip.write_audiofile(outputpath, logger=None)

  message = "Conversion and splitting successful."

  print(message)
import os
from tqdm import tqdm
def get_audio_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.mp4')]

audio_files = get_audio_files('audio')
print(audio_files)
for af in tqdm(audio_files):
    convert_mp4_to_wav_clips(af, 'MarcBotClips')

['audio/Trump Song Best Ever! Theme Song 2024!(A Trump Boy Can Survive)lets Go Brandon.mp4', 'audio/The Invisible Force - self-image – enables you to achieve great goals | Dan Lok | TEDxStanleyPark.mp4', 'audio/Mac Miller - Donald Trump.mp4', 'audio/Message from America.mp4', 'audio/Burden - Do Not Comply (Official Video).mp4', 'audio/LETS GO BRANDON - Theme Song - Loza Alexander - (OFFICIAL MUSIC VIDEO).mp4', 'audio/The Art of Letting Go | The Minimalists | TEDxFargo.mp4', 'audio/Pump The Trump.mp4', 'audio/The Secret of Becoming Mentally Strong | Amy Morin | TEDxOcala.mp4', 'audio/Tom MacDonald - "NO LIVES MATTER".mp4', 'audio/Why comfort will ruin your life | Bill Eckstrom | TEDxUniversityofNevada.mp4', 'audio/F Biden 2.mp4', 'audio/Getting stuck in the negatives (and how to get unstuck) | Alison Ledgerwood | TEDxUCDavis.mp4', 'audio/DONALD TRUMP SUPPORTERS RALLY SONG BY HENRY DAVIS.mp4', 'audio/Burden - Problem Child (Official Music Video).mp4', 'audio/Why we need pain to feel happ

  4%|▍         | 2/47 [00:00<00:02, 15.99it/s]

Trump_Song_Best_Ever!_Theme_Song_2024!(A_Trump_Boy_Can_Survive)lets_Go_Brandon
Conversion and splitting successful.
The_Invisible_Force_-_self-image_–_enables_you_to_achieve_great_goals_|_Dan_Lok_|_TEDxStanleyPark
Conversion and splitting successful.
Mac_Miller_-_Donald_Trump
Conversion and splitting successful.


  9%|▊         | 4/47 [00:00<00:02, 15.91it/s]

Message_from_America
Conversion and splitting successful.


...

Dear_Kyle_-_@Killwillc__ft
Conversion and splitting successful.
Let's_Go_Brandon_-_Bryson_Gray_(Feat
Conversion and splitting successful.
Burden_-_White_Privilege_(Official_Music_Video)
Conversion and splitting successful.
Living_without_shame:_How_we_can_empower_ourselves_|_Whitney_Thore_|_TEDxGreensboro
Conversion and splitting successful.


 98%|█████████▊| 46/47 [00:02<00:00, 15.82it/s]

Rewiring_how_you_look_at_yourself_|_Jenny_Schatzle_|_TEDxOaksChristianSchool
Conversion and splitting successful.
SOME_SAY_KOSM_-_YOUR_EYES_HAVE_YET_TO_OPEN_[OFFICIAL_MUSIC_VIDEO]_(2021)_SW_EXCLUSIVE
Conversion and splitting successful.
Choose_yourself_|_James_Altucher_|_TEDxSanDiego
Conversion and splitting successful.


100%|██████████| 47/47 [00:02<00:00, 15.92it/s]

This_Is_Not_Your_Practice_Life_|_AJ_Leon_|_TEDxFargo
Conversion and splitting successful.


In [6]:
!mkdir testfiles

mkdir: cannot create directory ‘testfiles’: File exists


In [7]:
from datasets import load_dataset
print("Loading Dataset")
# Select subset from MarcBotClips
audio_dataset = load_dataset("audiofolder", data_dir="MarcBotClips")["train"]

print("Normalizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {
        "original_sampling_rate": x["audio"]["sampling_rate"],
        "audio_array": normalize_waveform(
            torch.tensor(x["audio"]["array"]), x["audio"]["sampling_rate"]
        ),
    },
    remove_columns=["audio"],
    # keep_in_memory=True,
    writer_batch_size=15000,
)

print("Making sure the dataset is in the correct format")
# Making sure that the dataset consists only of 10 second clips at 16000 sample rate.
audio_dataset = audio_dataset.filter(
    lambda x: len(x["audio_array"][0]) ==  speech_tokenizer.sample_rate*LENGTH_OF_CLIPS,
)

print("Tokenizing the waveforms")
audio_dataset = audio_dataset.map(
    lambda x: {"tokens": tokenize(torch.tensor(x["audio_array"]))
               },
    remove_columns=[
        "audio_array",
    ],
    writer_batch_size=15000,
)

# Checking the files to see if the tokenization worked correctly.
for idx, t in enumerate(audio_dataset.select(range(0, 10))):
   save_to_file(torch.tensor(t["tokens"]).to(device), f"testfiles/{idx}_test.wav")

Loading Dataset


Resolving data files:   0%|          | 0/2372 [00:00<?, ?it/s]

Normalizing the waveforms


Map:   0%|          | 0/2372 [00:00<?, ? examples/s]

Making sure the dataset is in the correct format


Filter:   0%|          | 0/2372 [00:00<?, ? examples/s]

Tokenizing the waveforms


Map:   0%|          | 0/2329 [00:00<?, ? examples/s]

In [8]:
audio_dataset = audio_dataset.with_format('torch')
audio_dataset = audio_dataset.train_test_split(0.05)
# Setting up the train and test dataloader
train_dataloader = torch.utils.data.DataLoader(audio_dataset['train'], batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(audio_dataset['test'], batch_size=batch_size, shuffle=True)



In [9]:
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torchinfo
import torchaudio
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torch.nn import  functional as F
import wandb
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from armt.model import AssociativeMemoryCell, AssociativeRecurrentWrapper
from transformers import AutoModelForCausalLM, AutoConfig
import json

#hyperparams

epochs = 10
lr = 1e-3
block_size = 2000
n_segments = 2
device = "cuda" if torch.cuda.is_available() else "cpu"
max_iters = 10000
print_iters = 100
eval_iters = 10
eval_interval = 300
n_embed=384
n_heads = 6
n_layers = 6
dropout = 0.2
vocab_size=1024
from tqdm import tqdm

config = {
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "model_type": "gpt_neox",
  "vocab_size": vocab_size,
  "hidden_size": n_embed, 
  "num_hidden_layers": n_layers, 
  "num_attention_heads": n_heads, 
  "intermediate_size": n_embed * 4, 
  "max_position_embeddings": 2048,
  "bos_token_id": vocab_size - 2,
  "eos_token_id": vocab_size - 1,
  "hidden_act": "gelu",
  "rotary_pct": 0.25,
  "rotary_emb_base": 10000,
  "attention_dropout": 0.0,
  "hidden_dropout": 0.0,
  "classifier_dropout": dropout,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-5,
  "use_cache": True,
  "tie_word_embeddings": False,
  "use_parallel_residual": True
}
save_folder = "./configs/"
os.makedirs(save_folder, exist_ok=True)
save_path = save_folder + 'gptneox.json'
with open(save_path, 'w') as f:
    json.dump(config, f)
config = AutoConfig.from_pretrained(save_path)

mem_cell_args = dict(
    num_mem_tokens=16, 
    d_mem=64, 
    layers_attr="gpt_neox.layers",
)
rmt_config = dict(segment_size=block_size // n_segments, max_n_segments=n_segments)

*** Can't import RWKV model ***


In [10]:

def estimate_test_loss(model, dataset):
  model.eval()
  test_losses = []
  with torch.no_grad():
    for tokens in tqdm(dataset['test']['tokens']):
      tokens = tokens.to(device)
      x = tokens[:,:-1].contiguous()
      y = tokens[:,1:].contiguous()
      loss = model(input_ids=x,labels=y)['loss']
      test_losses.append(loss)
  model.train()
  return sum(test_losses)/len(test_losses)




def produce_wav(filename, model, example):
  firstbit = example.shape[-1]//2
  tokens = example[:firstbit]
  tokens = tokens.reshape(1,firstbit)
  max_new_tokens=example.shape[-1]-firstbit
  idx = tokens.to(device)
  idx_next = []
  for i in tqdm(range(max_new_tokens)):
    idx_cond = idx[:,-block_size:]
    logits = model(input_ids=idx_cond)['logits']
    last_timestep = logits[:,-1,:]
    probs = F.softmax(last_timestep, dim=1)
    next_index = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, next_index), dim=1)

  print(idx.shape,idx)
  # assert(torch.allclose(idx[0][:firstbit], example[:firstbit]))
  save_to_file(idx, f"{filename}_test.wav")
  save_to_file(example.unsqueeze(0), f"{filename}_input.wav")
  wandb.save(f"{filename}_test.wav")
  wandb.save( f"{filename}_input.wav")

def save_model_checkpoint(ind, model, dataset):
  model_path = f"checkpoints/{ind}_model.pt"
  torch.save(model.state_dict(), model_path)
  wandb.save(model_path)
  train_filename = f"checkpoints/{ind}_train"
  test_filename = f"checkpoints/{ind}_test"
  produce_wav(train_filename, model, dataset['train'][0]['tokens'][0])
  produce_wav(test_filename, model, dataset['test'][0]['tokens'][0])


In [12]:
import wandb
import time
import torchinfo
from tqdm.notebook import tqdm
wandb.init(project="speech_tokenizer_test")
base_model = AutoModelForCausalLM.from_config(config)
model = base_model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
print(torchinfo.summary(model))
model.train()
ind = 0
start_time = time.time()

for epoch in tqdm(range(epochs)):
  for batch in tqdm(train_dataloader):
    if batch['tokens'].shape[-1] != block_size:
      print("Emm....")
      continue
    batch_size = batch['tokens'].shape[0]
    tokens = batch['tokens'].to(device).reshape(batch_size,block_size)

    x = tokens[:,:-1].contiguous()
    y = tokens[:,1:].contiguous()
    out = model(input_ids=x,labels=y)
    logits, loss = out['logits'], out['loss']
    if ind % 10 == 0:
      print(loss)
      wandb.log({"Training Loss":loss.item(), "Iteration":ind, "Epoch":epoch, "time_elapsed":time.time()-start_time})
    if ind % 500 == 0:
      tl = estimate_test_loss(model, audio_dataset)
      print("testloss", tl)
      wandb.log({"Testing Loss":tl.item(), "Iteration":ind, "Epoch":epoch})
    if ind % 1000 == 0:
      save_model_checkpoint(ind, model, audio_dataset)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # Prevents gradient explosion.
    torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    ind += 1

model_path="overfittedmodel.pt"
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()
print(torchinfo.summary(model))



Layer (type:depth-idx)                             Param #
GPTNeoXForCausalLM                                 --
├─GPTNeoXModel: 1-1                                --
│    └─Embedding: 2-1                              393,216
│    └─Dropout: 2-2                                --
│    └─ModuleList: 2-3                             --
│    │    └─GPTNeoXLayer: 3-1                      1,774,464
│    │    └─GPTNeoXLayer: 3-2                      1,774,464
│    │    └─GPTNeoXLayer: 3-3                      1,774,464
│    │    └─GPTNeoXLayer: 3-4                      1,774,464
│    │    └─GPTNeoXLayer: 3-5                      1,774,464
│    │    └─GPTNeoXLayer: 3-6                      1,774,464
│    └─LayerNorm: 2-4                              768
│    └─GPTNeoXRotaryEmbedding: 2-5                 --
├─Linear: 1-2                                      393,216
Total params: 11,433,984
Trainable params: 11,433,984
Non-trainable params: 0


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/553 [00:00<?, ?it/s]

tensor(7.0214, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 0/117 [00:00<?, ?it/s]

...

torch.Size([1, 2000]) tensor([[ 942,  352,  245,  ..., 1019,  103,   75]], device='cuda:0')
tensor(2.8901, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6488, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.4708, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.1313, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6191, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.8046, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6791, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7068, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.6868, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.9418, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7845, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1164, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7196, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.4652, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2

  0%|          | 0/117 [00:00<?, ?it/s]

testloss tensor(3.3281, device='cuda:0')
tensor(2.8561, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7709, device='cuda:0', grad_fn=<NllLossBackward0>)


Epoch,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇█████
Iteration,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
Testing Loss,█▃▂▂▁▁▁▁▁▁▁▁
Training Loss,█▅▆▆▅▄▄▄▅▃▃▄▄▄▃▃▃▃▃▃▂▃▂▃▂▃▃▃▂▃▂▂▃▂▂▂▁▂▂▂
time_elapsed,▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
Epoch,9
Iteration,5520
Testing Loss,3.32812
Training Loss,2.77094
time_elapsed,365.8253


Layer (type:depth-idx)                             Param #
GPTNeoXForCausalLM                                 --
├─GPTNeoXModel: 1-1                                --
│    └─Embedding: 2-1                              393,216
│    └─Dropout: 2-2                                --
│    └─ModuleList: 2-3                             --
│    │    └─GPTNeoXLayer: 3-1                      1,774,464
│    │    └─GPTNeoXLayer: 3-2                      1,774,464
│    │    └─GPTNeoXLayer: 3-3                      1,774,464
│    │    └─GPTNeoXLayer: 3-4                      1,774,464
│    │    └─GPTNeoXLayer: 3-5                      1,774,464
│    │    └─GPTNeoXLayer: 3-6                      1,774,464
│    └─LayerNorm: 2-4                              768
│    └─GPTNeoXRotaryEmbedding: 2-5                 --
├─Linear: 1-2                                      393,216
Total params: 11,433,984
Trainable params: 11,433,984
Non-trainable params: 0


In [13]:
model_path="overfittedmodel.pt"
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()

Error: You must call wandb.init() before wandb.save()

In [ ]:
!mkdir generations/gpt_neox -p

In [19]:
def unconditional_generation(model):
    idx = torch.tensor([[10,]]).to(device)
    max_new_tokens = 1999
    idx_next = []
    for i in tqdm(range(max_new_tokens)):
        idx_cond = idx[:,-block_size:]
        out = model(idx_cond)
        logits = out['logits']
        last_timestep = logits[:,-1,:]
        probs = F.softmax(last_timestep, dim=1)
        next_index = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_index), dim=1)

    save_to_file(idx, f"unconditional_test.wav")

In [15]:
!mkdir testfiles/gpt_neox

In [16]:

for idx, t in enumerate(audio_dataset['test'].select(range(0, 10))):
   save_to_file(torch.tensor(t["tokens"]).to(device), f"testfiles/gpt_neox/{idx}_test.wav")

/tmp/ipykernel_2568121/3896174457.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  save_to_file(torch.tensor(t["tokens"]).to(device), f"testfiles/gpt_neox/{idx}_test.wav")


In [21]:
unconditional_generation(model)

  0%|          | 0/1999 [00:00<?, ?it/s]